# Analysis of microcompression on turgid and plasmolyzed BY2 cells

## Last Updated: December 19, 2021

This document was prepared by Leah Ginsberg, a member of the [Ravichandran Research Group](https://www.ravi.caltech.edu/) at [Caltech](http://www.caltech.edu) in collaboration with [Professor Eleftheria Roumeli](https://sites.google.com/uw.edu/roumeli-research-group/) from [University of Washington](https://www.washington.edu/). 

<img src="caltech_uw_logo.png">

*These instructions were generated from a Jupyter notebook.  You can download the notebook [here](microcompress_contact_finder.ipynb).*

In [1]:
import numpy as np               # general math operations
import pandas as pd              # pandas dataframes
import os                        # for opening .csv files in different folders
import altair as alt             # pretty plotting
import altair_catplot as altcat  # pretty box & whisker plots
from scipy import signal, stats, fftpack  # filtering and statistics
import matplotlib.pyplot as plt  # plotting histogram

alt.data_transformers.enable('data_server') # uses a background server for Altair plots (helps with performance for large data sets)

DataTransformerRegistry.enable('data_server')

In this notebook, we will utilize an iterative force thresholding method to calculate the point of contact between the microcompression sensor and the cell wall. This notebook requires the corrected displacement data generated in the notebook [microcompress_corr_displ.ipynb](microcompress_corr_displ.ipynb). Before attempting to run this notebook, ensure that you have the files `cell###_corr` in the microcompression data folder, which generate as the final step of the aforementioned notebook.

The contact finding steps are as follows:
1. Import the corrected data from microcompression experiments ($Z_{\text{corr}}$, $F$)
2. Calcuate the point of contact by iteratively offsetting force and lowering threshold
3. Fit line to initial portion of displacement curve

The fitted line is the initial stiffness of our BY-2 cells, which are presented in the box+whiskers plot in Figure 3 of our paper.

### 1. Import the corrected data from microcompression experiments ($Z_{\text{corr}}$, $F$)

As before, we will load data stored on disk into a Python data structure using `pandas`. If necessary, change the first line to specify the path where the files are contained on your local computer.

In [2]:
path = 'microcompress_data'
dir_list = os.listdir(path) # prints a list of strings with all folder names in the specified path

# folder names for each drug treatment
sorbitol_folderName = path + '/BY2_in_sorbitol/'
C2_folderName = path + '/BY2_in_C2/'
ory_folderName = path + '/BY2_in_oryzalin/'
LatB_folderName = path + '/BY2_in_LatB/'
ory_sorbitol_folderName = path + '/BY2_in_oryzalin_sorbitol/'
LatB_sorbitol_folderName = path + '/BY2_in_LatB_sorbitol/'

# combine all the lists of folder names into one list
folderNames = [C2_folderName, ory_folderName, LatB_folderName, sorbitol_folderName, ory_sorbitol_folderName, LatB_sorbitol_folderName]

# print everything to make sure it worked
print(folderNames)

['microcompress_data/BY2_in_C2/', 'microcompress_data/BY2_in_oryzalin/', 'microcompress_data/BY2_in_LatB/', 'microcompress_data/BY2_in_sorbitol/', 'microcompress_data/BY2_in_oryzalin_sorbitol/', 'microcompress_data/BY2_in_LatB_sorbitol/']


Next, we need to determine which files are the corrected sample data.

In [3]:
# Get names of all files to be used as calibration data or sample data
sample_fileNames = []
for folderName in folderNames:
    dir_list = os.listdir(folderName)
    for fileName in dir_list:
        if fileName.startswith('cell') and ('_corr' in fileName) and not ('.' in fileName):
            sample_fileNames.append(folderName + fileName)
            
# Check how many files it found
len(sample_fileNames)

97

If all went right, you should have 97 entries in `sample_fileNames`. Now, let's load the sample data into a dataframe.

In [4]:
# Organize dataframe by column with additional information about experiment not found in .csv file (treatment, plasmolysis, etc.)
df_sample = pd.DataFrame(columns=['Index','Time (s)','Displacement (um)','Pos X (um)','Pos Y (um)','Pos Z (um)','Force A (uN)','Force B (uN)','fileName','indenting','plasmolyzed','treatment', 'Corrected Displacement (um)'])

# loop through all sample data
for j, fileName in enumerate(sample_fileNames):
    
    # read data from file to dataframe
    df = pd.read_csv(fileName, header=3, sep='\t', names=['Index','Time (s)','Displacement (um)','Pos X (um)','Pos Y (um)','Pos Z (um)','Force A (uN)','Force B (uN)','fileName','indenting','plasmolyzed','treatment', 'Corrected Displacement (um)'])
    
    df['fileName']=fileName
    df['indenting']=df['Index']==1.001 # in data files, Index 1.001 indicates indenting data, and Index 2.001 indicates retraction data
    
    # Take information about experiment from filename and put into dataframe
    if 'sorbitol' in fileName:
        df['plasmolyzed'] = True
    else:
        df['plasmolyzed'] = False
    
    if 'LatB' in fileName:
        df['treatment'] = 'LatB'
    elif 'oryzalin' in fileName:
        df['treatment'] = 'oryzalin'
    else:
        df['treatment'] = 'C2'
    
    # concatenate all sample dataframes into one dataframe
    df_sample = pd.concat([df_sample, df])

df_sample = df_sample.reset_index(drop=True) # resets row index to start from 0
    
# Take a look at the dataframe we created
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,plasmolyzed,treatment,Corrected Displacement (um)
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,False,C2,0.362238
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,False,C2,0.384944
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,False,C2,0.409157
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,False,C2,0.445188
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,False,C2,0.485652


This data is in a format that is known as **tidy**. See this [paper](https://www.jstatsoft.org/article/view/v059i10) by Hadley Wickam for a detailed discussion of tidying data. The basic structure of tidy data is:
1. Each variable is a column
2. Each observation is a row
3. Each type of observational unit is its own table

With our data set up in this way, we can easily find data, say, with a specific treatment using Boolean slicing.

One more piece of information we'd like to have in the `DataFrame` is a column indicating whether or not the maximum indentation force is above 700 uN. These data sets are considered **Full Indentation**s, and will be used in the dissipated energy calculations later.

In [5]:
df_sample.insert(10,'Full Indentation', False, False) # initialize full indentation indicator column

# See what that looks like
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,Full Indentation,plasmolyzed,treatment,Corrected Displacement (um)
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,False,False,C2,0.362238
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,False,False,C2,0.384944
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,False,False,C2,0.409157
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,False,False,C2,0.445188
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,False,False,C2,0.485652


*Note that if you run the above block of code more than once, you will get a* `ValueError`.

In [6]:
for fileName in df_sample['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName]
    if max(df['Force A (uN)'])>700:
        df_sample.loc[df_sample['fileName']==fileName, 'Full Indentation'] = True
    else:
        df_sample.loc[df_sample['fileName']==fileName, 'Full Indentation'] = False

# Check that it worked
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,Full Indentation,plasmolyzed,treatment,Corrected Displacement (um)
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.362238
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.384944
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.409157
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.445188
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.485652


### 2. Use threshold data to calcuate the point of contact

We follow the methodology of Routier-Kierzkowska et al. (2012) to correct our data for the sensor stiffness. Quoting the methods section of their paper "Cellular Force Microscopy for in Vivo Measurements of Plant Tissue Mechanics":

>"For each measurement cycle, the force signal is offset, so that the mean force signal before the contact point is equal to zero. The contact point is then determined to be the closest point to the maximal indentation that is below a user-defined contact force threshold (usually smaller than 1 $\mu$N). Both force offsetting and contact point detection are performed iteratively, using finer and finer threshold values. Indentation depth is then given by the sensor position corrected for sensor deflection) minus the contact point position."

We have iteratively examined each of the 98 data sets to determine the appropriate threshold, and those values are saved in a file called `thresholds.csv`, which is included in the Github repository. In the next step, we'll open the threshold values and use them to correct the indentation data.

In [7]:
df_sample.insert(14,'offset force A (uN)', df_sample['Force A (uN)'],False) # initialize offset force column

# Check that it worked
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,Full Indentation,plasmolyzed,treatment,Corrected Displacement (um),offset force A (uN)
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.362238,-10.74826
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.384944,-10.73709
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.409157,-10.72752
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.445188,-10.73550
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.485652,-10.34471


*Note that if you run the above block of code more than once, you will get a* `ValueError`.

For now, the **offset force A (uN)** and **Corrected Displacement (um)** columns are just copies of the **Force A (uN)** and **Displacement (um)** columns, respectively. 

In the next step, you may need to correct the path specified in the argument of `pd.read_csv()` to reflect the location of the files on your local computer.

In [8]:
# import threshold data from csv file
df_thresholds = pd.read_csv('microcompress_data/thresholds.csv', names=['folderName','fileName','threshold'], sep='\s+')
df_thresholds.head()

,folderName,fileName,threshold
0,BY2_in_C2,cell001,1.0
1,BY2_in_C2,cell002,1.0
2,BY2_in_C2,cell003,0.5
3,BY2_in_C2,cell004,1.0
4,BY2_in_C2,cell005,10.0


We're going to copy the threshold value that corresponds to each file and put it in a new column, called **threshold**.

In [9]:
fileNames = path + '/' + df_thresholds['folderName'] + '/' + df_thresholds['fileName']

for fileName in fileNames:
    # copy threshold data to df_sample
    df_thresh = df_thresholds[(df_thresholds['folderName']==fileName.split('/')[-2]) & (df_thresholds['fileName']==fileName.split('/')[-1])].reset_index(drop=True)
    df_sample.loc[df_sample['fileName']==fileName+'_corr', 'threshold'] = df_thresh['threshold'][0]
    
# Check that it worked
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,Full Indentation,plasmolyzed,treatment,Corrected Displacement (um),offset force A (uN),threshold
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.362238,-10.74826,1.0
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.384944,-10.73709,1.0
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.409157,-10.72752,1.0
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.445188,-10.73550,1.0
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.485652,-10.34471,1.0


In [10]:
for fileName in df_sample['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName].copy()        # create copy of current data set
    end_thr = df['threshold'].unique()[0]                         # optimal final threshold value

    # sequence of thresholds depending upon final threshold value
    if end_thr>=10.0:
        thresholds = [20,end_thr]
    elif end_thr>=5.0:
        thresholds = [20, 10, end_thr]
    else:
        thresholds = [20, 10, 5, end_thr]

    # loop over threshold values
    for i, threshold in enumerate(thresholds):
        df_indenting = df[df['indenting']]                                            # only concerned with indentation data
        df_precontact = df_indenting[df_indenting['offset force A (uN)']<threshold]   # designate all data below threshold as 'precontact'
        force_offset = np.mean(df_precontact['offset force A (uN)'])                  # force offset is mean of precontact data (so that new mean of precontact data is zero)
        df.loc[:,'offset force A (uN)'] = df['offset force A (uN)'] - force_offset    # write new offset force on dataframe
        # Find the contact point (the point where the force data crosses the threshold)
        contact_point = max(df_indenting[df_indenting['offset force A (uN)']<threshold]['Corrected Displacement (um)'])
        # get rid of initial data if needed
        if contact_point>20:
            df = df[df['Corrected Displacement (um)']>contact_point-20]
            df.loc[:,'Corrected Displacement (um)'] = df['Corrected Displacement (um)']-(contact_point-20)
        df.loc[:,'indentation depth (um)'] = df['Corrected Displacement (um)'] - contact_point
        # save results in dataframe
        df_sample.loc[df_sample['fileName']==fileName,'Corrected Displacement (um)'] = df['Corrected Displacement (um)']
        df_sample.loc[df_sample['fileName']==fileName,'indentation depth (um)'] = df['indentation depth (um)']
        df_sample.loc[df_sample['fileName']==fileName,'offset force A (uN)'] = df['offset force A (uN)']

df_sample = df_sample.dropna() # drop rows with nan
        
# Take a look
df_sample.head()

,Index,Time (s),Displacement (um),Pos X (um),Pos Y (um),Pos Z (um),Force A (uN),Force B (uN),fileName,indenting,Full Indentation,plasmolyzed,treatment,Corrected Displacement (um),offset force A (uN),threshold,indentation depth (um)
0,1.001,0.466,0.32000,184.00625,-106.00100,49.65400,-10.74826,0.00253,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.362238,-0.544755,1.0,-7.982426
1,1.001,0.595,0.34275,184.00600,-106.00075,49.63125,-10.73709,0.00397,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.384944,-0.533585,1.0,-7.959720
2,1.001,0.736,0.36700,184.00700,-106.00050,49.60700,-10.72752,0.00317,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.409157,-0.524015,1.0,-7.935508
3,1.001,0.851,0.40300,184.00650,-106.00025,49.57100,-10.73550,0.00429,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.445188,-0.531995,1.0,-7.899476
4,1.001,0.948,0.44500,184.00650,-106.00000,49.52900,-10.34471,0.00795,microcompress_data/BY2_in_C2/cell001_corr,True,True,False,C2,0.485652,-0.141205,1.0,-7.859012


Done! Now, plotting **Force A (uN)** against **indentation depth (um)** will show the indentation data, corrected for sensor stiffness, and adjusted such that zero on the x-axis corresponds to the initial point of contact between the sensor and the cell wall. Let's see cell001 as an example.

In [11]:
corr_data = alt.Chart(df_sample[df_sample['fileName']=='microcompress_data/BY2_in_C2/cell001_corr']).mark_circle().encode(
    x='Corrected Displacement (um)',
    y='Force A (uN)',
    color=alt.value('black')
)

ind_data = alt.Chart(df_sample[df_sample['fileName']=='microcompress_data/BY2_in_C2/cell001_corr']).mark_circle().encode(
    x='indentation depth (um)',
    y='offset force A (uN)',
    color=alt.value('red')
)

(corr_data + ind_data).configure_axis(grid=True,
    labelFontSize=15,
    titleFontSize=15).interactive()

alt.LayerChart(...)

The origin (0,0) on the new data set (plotted in red) corresponds to the contact point. Soon after contact, the measured reaction force from contact forces between the sensor and the cell begin to increase.

One interesting statistic we can now look at is the average indentation depth for cells that were fully indented (maximum force went <700uN).

In [12]:
max_ind_depth = np.zeros(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))
for i, fileName in enumerate(df_sample[df_sample['Full Indentation']]['fileName'].unique()):
    df = df_sample[df_sample['fileName']==fileName].copy()
    max_ind_depth[i] = max(df['indentation depth (um)'])
# print standard error
mean_ind = np.mean(max_ind_depth)
stderr_ind = np.std(max_ind_depth)/np.sqrt(len(max_ind_depth))
print('The average total indentation depth in an experiment is {:.2f} +/- {:.2f} um.'.format(mean_ind, stderr_ind))

The average total indentation depth in an experiment is 12.35 +/- 0.70 um.


If **Full Indentation** truly meant that the cells were compressed fully, to the point where one cell wall touched the other and were compressed into the glass slide, then this number would correspond to the diameter of the cells. However, in our experiments some cells (especially the turgid ones in C2) were observed that could withstand more than 900uN and still appear intact in the optical microscope. 

Finally, we'll take an example of a turgid and of a plasmolyzed cell and plot the force-indentation data for presentation in our paper.

In [14]:
# chart for figure 3A in paper
alt.Chart(df_sample[(df_sample['indentation depth (um)']>0)&((df_sample['fileName']=='microcompress_data/BY2_in_sorbitol/cell072_corr')|(df_sample['fileName']=='microcompress_data/BY2_in_C2/cell001_corr'))]).mark_circle().encode(
    x='indentation depth (um)',
    y='offset force A (uN)',
    color = alt.Color('fileName:N', scale=alt.Scale(domain=['microcompress_data/BY2_in_C2/cell001_corr', 'microcompress_data/BY2_in_sorbitol/cell072_corr'], range=['red', 'blue']))
).configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).properties(
    width=200,
    height=300
)

alt.Chart(...)

### 3. Filter force data from all samples using Savitsky-Golay filter

The Savitzky-Golay filter is a digital filter that fits a moving window of data points to a low-order polynomial. In this case, I am fitting 5 data points at a time to a second-order polynomial. The [Wikipedia page](https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter) has a nice animation illustrating how this moving window works. This step eliminates the high frequency noise from our experimental data.

In [15]:
fileName = 'microcompress_data/BY2_in_sorbitol/cell072_corr'
df = df_sample[df_sample['fileName']==fileName]

# parameters for Savitzky-Golay filter
window_length = 25 # number of data points in moving window at any given time in filter
polyorder = 2 # order of polynomial to fit in filter
  
# reset row numbers
df = df.reset_index(drop=True)
    
# separate indenting and retraction data for filter
df_indent = df[df['indenting']]
df_retract = df[~df['indenting'].astype('bool')]
    
# filter indenting and retraction data
f_filt_ind = signal.savgol_filter(df_indent['offset force A (uN)'], window_length, polyorder) 
f_filt_ret = signal.savgol_filter(df_retract['offset force A (uN)'], window_length, polyorder)
    
# copy filtered data onto appropriate slices of dataframe
df.loc[df['indenting'],'filtered force A (uN)'] = f_filt_ind
df.loc[~df['indenting'].astype('bool'),'filtered force A (uN)'] = f_filt_ret

unfiltered_chart = alt.Chart(df[df['indenting']]).mark_circle().encode(
    x=alt.X('indentation depth (um)',scale=alt.Scale(domain=(0.0,6.5))),
    y=alt.Y('offset force A (uN)',scale=alt.Scale(domain=(-7.0,20.0))),
    color=alt.value('black')
)
filtered_chart = alt.Chart(df[df['indenting']]).mark_square().encode(
    x=alt.X('indentation depth (um)',scale=alt.Scale(domain=(0.0,6.5))),
    y=alt.Y('filtered force A (uN)',scale=alt.Scale(domain=(-7.0,20.0))),
    color=alt.value('green')
)
(unfiltered_chart+filtered_chart).configure_axis(grid=True,
    labelFontSize=15,
    titleFontSize=15).interactive()

alt.LayerChart(...)

You can scroll and drag the chart above to compare the filtered (green squares) and unfiltered data (black circles) in an example. In the next block of code, we'll repeat this filtering on all data sets in a loop. 

In [19]:
# loop through each data set and run Savitzky-Golay filter
df_sample['filtered force A (uN)'] = 0 # initialize new column in dataframe
window_length = 25 # number of data points in moving window at any given time in filter
polyorder = 2 # order of polynomial to fit in filter
for fileName in df_sample['fileName'].unique():
    # separate by experiment label
    df = df_sample[df_sample['fileName']==fileName]
    
    # reset row numbers and set initial force to 0.0
    df = df.reset_index(drop=True)
    df['offset force A (uN)'] = df['offset force A (uN)'] - df['offset force A (uN)'][0] 
    
    # separate indenting and retraction data for filter
    df_indent = df[df['indenting']]
    df_retract = df[~df['indenting'].astype('bool')]
    
    # filter indenting and retraction data
    f_filt_ind = signal.savgol_filter(df_indent['offset force A (uN)'], window_length, polyorder)
    # copy filtered data onto appropriate slices of dataframe
    df_sample.loc[(df_sample['fileName']==fileName)&(df_sample['indenting']),'filtered force A (uN)'] = f_filt_ind
    if len(df_retract)!=0:
        # only filter if retraction data exists
        f_filt_ret = signal.savgol_filter(df_retract['offset force A (uN)'], window_length, polyorder)
        # copy filtered data onto appropriate slices of dataframe
        df_sample.loc[(df_sample['fileName']==fileName)&(~df_sample['indenting'].astype('bool')),'filtered force A (uN)'] = f_filt_ret

# take a look
alt.Chart(df_sample[df_sample['indenting']]).mark_square().encode(
    x='indentation depth (um)',
    y='filtered force A (uN)',
    color = 'treatment:N'
).interactive()

alt.Chart(...)

In [ ]:
# df_sample[df_sample['fileName']=='BY2_in_sorbitol/cell072_corr'].to_csv('cell072_proc')

### 4. Fit line to initial portion of displacement curve

Like we did in the contact finding section, we'll first take a look at one example case, and then apply the same procedure to all data sets in a loop.

In [ ]:
fileName = 'BY2_in_water/cell150_corr'
df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN
name = df['fileName'][0]

# only take indentation data
df = df[df['indenting']]

# set initial force and displacement to (0,0)
df = df[df['indentation depth (um)']>0.0].reset_index(drop=True)
df['filtered force A (uN)'] = df['filtered force A (uN)'] - df['filtered force A (uN)'][0] 

# take first 1um of displacement after touching cell if over cell wall
df_init = df[(df['indentation depth (um)']<1.0)&(df['indentation depth (um)']>=0.0)]

# Now perform a linear fit
f = np.polyfit(df_init['indentation depth (um)'], df_init['filtered force A (uN)'], 1)

# Add the linear fit to the plot
displ = df_init['indentation depth (um)']
force = f[0]*displ + f[1]
kref = f[0]
treatment = df['treatment'][0]
names = name
df_linfit = pd.DataFrame({'indentation depth (um)': displ, 'offset force A (uN)': force})
print(name, f[0])

df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN

# Take a look
chart_data = alt.Chart(df).mark_square().encode(
    x=alt.X('indentation depth (um)'), # scale=alt.Scale(domain=(0,2.0))),
    y=alt.Y('offset force A (uN)'), # scale=alt.Scale(domain=(0,10.0))),
    color = alt.value('green')
)
chart_fit = alt.Chart(df_linfit).mark_line().encode(
    x=alt.X('indentation depth (um)'), # scale=alt.Scale(domain=(0,2.0))),
    y=alt.Y('offset force A (uN)'), # scale=alt.Scale(domain=(0,10.0))),
    color=alt.value('black')
)
(chart_data + chart_fit).configure_axis(grid=True,
    labelFontSize=15,
    titleFontSize=15).interactive()

In [ ]:
kref = np.empty(len(df_sample['fileName'].unique()))
deltaF = np.empty(len(df_sample['fileName'].unique()))
treatment = ['' for x in range(len(df_sample['fileName'].unique()))]
names = ['' for x in range(len(df_sample['fileName'].unique()))]
locations = ['' for x in range(len(df_sample['fileName'].unique()))]
details = ['' for x in range(len(df_sample['fileName'].unique()))]
maxF = np.empty(len(df_sample['fileName'].unique()))
df_linfit_concat = pd.DataFrame(columns=['offset force A (uN)','indentation depth (um)'])
i=0

for fileName in df_sample['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN

    # only take indentation data
    df = df[df['indenting']]
    
    # set initial force and displacement to (0,0)
    df = df[df['indentation depth (um)']>0.0].reset_index(drop=True)
    df['filtered force A (uN)'] = df['filtered force A (uN)'] - df['filtered force A (uN)'][0] 

    # if contact detected, find initial stiffness
    if max(df['indentation depth (um)'])>0.0:

        # take first 1um of displacement after touching cell if over cell wall
        df_glass = df[(df['indentation depth (um)']<1.0)&(df['indentation depth (um)']>=0.0)]

        # Now perform a linear fit
        f = np.polyfit(df_glass['indentation depth (um)'], df_glass['filtered force A (uN)'], 1)
        
        # Add the linear fit to the plot
        displ = df_glass['indentation depth (um)']
        force = f[0]*displ + f[1]
        maxF[i] = max(df['offset force A (uN)'])
        kref[i] = f[0]
        treatment[i] = df['treatment'][0]
        locations[i] = df['location'][0]
        details[i] = df['details'][0]
        names[i] = fileName
        df_linfit = pd.DataFrame({'indentation depth (um)': displ, 'filtered force A (uN)': force})
        df_linfit_concat = pd.concat([df_linfit, df_linfit_concat], sort=False)
        
        # measure "strength" by measuring delta force required for 1.0um of indentation
        deltaF[i] = max(df[df['indentation depth (um)']<1.0]['filtered force A (uN)'])
    
    # if contact not detected, fill in data for df_kref
    else:
        kref[i] = 0.0
        maxF[i] = max(df['offset force A (uN)'])
        treatment[i] = 'no contact detected'
        locations[i] = df['location'][0]
        names[i] = fileName
        # measure "strength" by measuring delta force required for 1.0um of indentation
        deltaF[i] = max(df[df['indentation depth (um)']<1.0]['filtered force A (uN)'])
        
    if deltaF[i]>700:
        print(fileName)
    i+=1
    
df_kref = pd.DataFrame({'name':names, 'max force (uN)': maxF, 'location':locations, 'kinit': kref, 'deltaF':deltaF, 'treatment': treatment, 'details': details})

In [ ]:
# // box+jitter plot with outliers removed
# the box_mark argument sets the style of the bar
# the whisker_mark argument sets the style for the error bar
# (to select different colors, you'll want to google "hex color code")
# for mark type, you can use point, circle, square, etc. (not sure how to pick the size just yet)
altcat.catplot(df_kref[(df_kref['kinit']>0)&(df_kref['kinit']<100)&((df_kref['treatment']=='water')|(df_kref['treatment']=='oryzalin')|(df_kref['treatment']=='LatB')|(df_kref['treatment']=='C2'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kinit:Q',                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
# // box+jitter plot with outliers removed
# the box_mark argument sets the style of the bar
# the whisker_mark argument sets the style for the error bar
# (to select different colors, you'll want to google "hex color code")
# for mark type, you can use point, circle, square, etc. (not sure how to pick the size just yet)
altcat.catplot(df_kref[(df_kref['kinit']>0)&((df_kref['treatment']=='sorbitol')|(df_kref['treatment']=='oryzalin+sorbitol')|(df_kref['treatment']=='LatB+sorbitol'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kinit:Q',                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
df_kref[df_kref['treatment']=='sorbitol']

In [ ]:
# plasmolyzed results from CW
altcat.catplot(df_kref[(df_kref['kinit']>0)&(df_kref['location']=='cyto')&((df_kref['treatment']=='sorbitol')|(df_kref['treatment']=='oryzalin+sorbitol')|(df_kref['treatment']=='LatB+sorbitol'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kinit:Q',                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
# plasmolyzed results from CW
altcat.catplot(df_kref[(df_kref['kinit']>0)&(df_kref['treatment']=='sorbitol')],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('location:N'),
                             x='kinit:Q',                             color=alt.Color('location:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
# box+jitter plot with outliers removed
altcat.catplot(df_kref[df_kref['kinit']>0],
               height=250,
               width=450,
               mark='point',
               encoding=dict(y=alt.Y('treatment:N'),
                             x='deltaF:Q',
                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
fileName = 'BY2_in_sorbitol/cell073_corr'
df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN
name = df['fileName'][0]

# only take retraction data
df = df[~df['indenting'].astype('bool')]

# set initial force and displacement to (0,0)
df = df[df['indentation depth (um)']>0.0].reset_index(drop=True)
df['filtered force A (uN)'] = df['filtered force A (uN)'] - df['filtered force A (uN)'].iloc[-1] 

# take first 1um of displacement after reaching max force
df_init = df[(df['indentation depth (um)']>max(df['indentation depth (um)'])-1)]

# Now perform a linear fit
f = np.polyfit(df_init['indentation depth (um)'], df_init['filtered force A (uN)'], 1)

# Add the linear fit to the plot
displ = df_init['indentation depth (um)']
force = f[0]*displ + f[1]
kref = f[0]
treatment = df['treatment'][0]
names = name
df_linfit = pd.DataFrame({'indentation depth (um)': displ, 'offset force A (uN)': force})
print(name, f[0])

df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN

# Take a look
chart_data = alt.Chart(df).mark_square().encode(
    x=alt.X('indentation depth (um)'), # scale=alt.Scale(domain=(0,2.0))),
    y=alt.Y('filtered force A (uN)'), # scale=alt.Scale(domain=(0,10.0))),
    color = alt.value('green')
)
chart_fit = alt.Chart(df_linfit).mark_line().encode(
    x=alt.X('indentation depth (um)'), # scale=alt.Scale(domain=(0,2.0))),
    y=alt.Y('offset force A (uN)'), # scale=alt.Scale(domain=(0,10.0))),
    color=alt.value('black')
)
(chart_data+chart_fit).configure_axis(grid=True,
    labelFontSize=15,
    titleFontSize=15).interactive()

In [ ]:
kref = np.empty(len(df_sample['fileName'].unique()))
treatment = ['' for x in range(len(df_sample['fileName'].unique()))]
names = ['' for x in range(len(df_sample['fileName'].unique()))]
maxF = np.empty(len(df_sample['fileName'].unique()))
df_linfit_concat = pd.DataFrame(columns=['offset force A (uN)','indentation depth (um)'])
i=0

for fileName in df_sample[df_sample['Full Indentation']]['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN

    # only take retraction data
    df = df[~df['indenting'].astype('bool')]
    
    # set initial force and displacement to (0,0)
    df = df[df['indentation depth (um)']>0.0].reset_index(drop=True)
    df['filtered force A (uN)'] = df['filtered force A (uN)'] - df['filtered force A (uN)'].iloc[-1] 

    # if contact detected, find initial stiffness
    if max(df['indentation depth (um)'])>0.0:
        
        # take first 1um of displacement after reaching max force
        df_glass = df[(df['indentation depth (um)']>max(df['indentation depth (um)'])-1)]

        # Now perform a linear fit
        f = np.polyfit(df_glass['indentation depth (um)'], df_glass['filtered force A (uN)'], 1)
        
        # Add the linear fit to the plot
        displ = df_glass['indentation depth (um)']
        force = f[0]*displ + f[1]
        maxF[i] = max(df['offset force A (uN)'])
        kref[i] = f[0]
        treatment[i] = df['treatment'].iloc[-1]
        names[i] = fileName
        df_linfit = pd.DataFrame({'indentation depth (um)': displ, 'filtered force A (uN)': force})
        df_linfit_concat = pd.concat([df_linfit, df_linfit_concat], sort=False)
           
    # if contact not detected, fill in data for df_kref
    else:
        kref[i] = 0.0
        maxF[i] = max(df['offset force A (uN)'])
        treatment[i] = 'no contact detected'
        locations[i] = df['location'][0]
        names[i] = fileName
    
    df_kref.loc[df_kref['name']==fileName, 'kret'] = kref[i]
    i+=1
    
df_kref.head()

In [ ]:
altcat.catplot(df_kref[(df_kref['max force (uN)']>700)&((df_kref['treatment']=='water')|(df_kref['treatment']=='oryzalin')|(df_kref['treatment']=='LatB')|(df_kref['treatment']=='C2'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kret:Q',
                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
altcat.catplot(df_kref[(df_kref['max force (uN)']>700)&((df_kref['treatment']=='sorbitol')|(df_kref['treatment']=='oryzalin+sorbitol')|(df_kref['treatment']=='LatB+sorbitol'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kret:Q',
                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
# plasmolyzed results from CW
altcat.catplot(df_kref[(df_kref['kinit']>0)&(df_kref['location']=='CW')&((df_kref['treatment']=='sorbitol')|(df_kref['treatment']=='oryzalin+sorbitol')|(df_kref['treatment']=='LatB+sorbitol'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='kret:Q',                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

### (9) Calculate area between indenting and unindenting curves

In turgid cells, we can calculate the energy dissipated by taking the area between the indenting and retraction curves for full indentation (max force greater than 700 uN). In plasmolyzed cells, this amount of force will permanently damage the cells, so the energy dissipation measurement has to come from a lower force threshold (we'll use measurements with max force between 350 and 500 uN.

In [ ]:
fileName='BY2_in_sorbitol/cell103_corr'
df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN
name = df['fileName'][0]

alt.Chart(df).mark_point().encode(
    x=alt.X('indentation depth (um)'),
    y=alt.Y('offset force A (uN)'),
    color = 'indenting').configure_axis(grid=True,
    labelFontSize=15,
    titleFontSize=15)

In [ ]:
displ = df[df['indentation depth (um)']>0]['indentation depth (um)'].reset_index(drop=True)
force = df[df['indentation depth (um)']>0]['offset force A (uN)'].reset_index(drop=True)

In [ ]:
# Area of a polygon
# A = | 0.5 * (x1*y2 - y1*x2 + x2*y3 - y2*x3 + ... + xn*y1 - yn*x1) |

A = 0
for i in range(len(displ)):
    if i==len(displ)-1:
        A += displ[i] * force[0] - force[i] * displ[0]
    else:
        A += displ[i] * force[i+1] - force[i] * displ[i+1]
A = abs(A*0.5)
print(A)

Let's try to repeat this procedure on all data sets of full indentation (where the maximum force went above 700uN) in turgid cells, and plasmolyzed cells with a max force between 350 and 500 uN.

In [ ]:
# Initialize vectors for loop
area = np.empty(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))
treatment = ['' for x in range(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))]
names = ['' for x in range(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))]
zero_displ = np.empty(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))
j=0

for fileName in df_sample[df_sample['Full Indentation']]['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN
    zero_displ[j] = df[df['indentation depth (um)']>0]['Pos Z (um)'].reset_index(drop=True)[0]
    displ = df[df['indentation depth (um)']>0]['indentation depth (um)'].reset_index(drop=True)
    force = df[df['indentation depth (um)']>0]['offset force A (uN)'].reset_index(drop=True)
    A = 0
    for i in range(len(displ)):
        if i==len(displ)-1:
            A += displ[i] * force[0] - force[i] * displ[0]
        else:
            A += displ[i] * force[i+1] - force[i] * displ[i+1]
    area[j] = abs(A*0.5)
    names[j] = fileName
    treatment[j] = df['treatment'][0]
    
    df_kref.loc[df_kref['name']==fileName, 'area'] = area[j]
    df_kref.loc[df_kref['name']==fileName, 'Pos Z zero (um)'] = zero_displ[j]
    j+=1
df_kref.head()

In [ ]:
# Initialize vectors for loop
area = np.empty(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))
treatment = ['' for x in range(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))]
names = ['' for x in range(len(df_sample[df_sample['Full Indentation']]['fileName'].unique()))]
j=0

for fileName in df_sample[df_sample['Full Indentation']]['fileName'].unique():
    df = df_sample[df_sample['fileName']==fileName].reset_index(drop=True) # need to reset index since we dropped rows with NaN
    
    displ = df[(df['indentation depth (um)']>0)&(df['offset force A (uN)']>50)]['indentation depth (um)'].reset_index(drop=True)
    force = df[(df['indentation depth (um)']>0)&(df['offset force A (uN)']>50)]['offset force A (uN)'].reset_index(drop=True)
    A = 0
    for i in range(len(displ)):
        if i==len(displ)-1:
            A += displ[i] * force[0] - force[i] * displ[0]
        else:
            A += displ[i] * force[i+1] - force[i] * displ[i+1]
    area[j] = abs(A*0.5)
    names[j] = fileName
    treatment[j] = df['treatment'][0]
    
    df_kref.loc[df_kref['name']==fileName, 'area'] = area[j]
    j+=1
np.std(df_kref[df_kref['treatment']=='sorbitol']['area'].dropna())

In [ ]:
# remove water cases with a lot of sliding
df_kref = df_kref[(df_kref['name']!='BY2_in_water/cell124_corr')&(df_kref['name']!='BY2_in_water/cell139_corr')&(df_kref['name']!='BY2_in_water/cell191_corr')&(df_kref['name']!='BY2_in_water/cell192_corr')]

In [ ]:
altcat.catplot(df_kref[(df_kref['max force (uN)']>700)&((df_kref['treatment']=='water')|(df_kref['treatment']=='oryzalin')|(df_kref['treatment']=='LatB')|(df_kref['treatment']=='C2'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='area:Q',
                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
altcat.catplot(df_kref[(df_kref['max force (uN)']>700)&(df_kref['location']=='cyto')&((df_kref['treatment']=='sorbitol')|(df_kref['treatment']=='oryzalin+sorbitol')|(df_kref['treatment']=='LatB+sorbitol'))],
               height=250,
               width=450,
               mark=dict(opacity=1.0, type='circle'),
               box_mark=dict(strokeWidth=2, opacity=0.5, color='#eeeeee'),
               whisker_mark=dict(strokeWidth=2, opacity=0.5, color='#bababa'),
               encoding=dict(y=alt.Y('treatment:N'),
                             x='area:Q',
                             color=alt.Color('treatment:N'),
                             tooltip=alt.Tooltip(['name:N'], title='name')),
               transform='jitterbox'
              ).configure_text(
                  fontWeight='bold', fontSize=15
              ).configure_legend(labelFontSize=15,
    titleFontSize=15).configure_axis(labelFontSize=15, titleFontSize=15)

In [ ]:
# print results to csv
df_kref.to_csv('BY2_stiffnesses',index=False)